In [27]:
import numpy as np
import pandas as pd
import datetime as dt
from collections import Counter

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d, Axes3D # VERY important to have capitalization
import seaborn as sns
import re

# Turn off warnings
import warnings
warnings.filterwarnings('ignore')

# Pandas options
pd.set_option("max_rows", 30)
pd.set_option("max_columns", None)
pd.set_option("precision", 3)

import geocoder
import gmaps
import gmaps.datasets

with open(r'/home/harrisonized/Desktop/gmaps_apikey.txt') as f: # Grab API Key
    api_key = f.readline()
    f.close
    
gmaps.configure(api_key=api_key) # Fill in API Key

# Working with hyperlinks
import bs4
import urllib3
from urllib.request import urlopen

In [2]:
# Function that formats the turnstile_data
# Run after the turnstile file import

def format_turnstile_df(turnstile_csv_df):

    """
    turnstile_csv_df is the dataframe of the imported turnstile.csv file
    Warning: Run once per file ONLY. The function will throw an error if run twice on the same file.
    """

    global turnstile_df
    turnstile_df = turnstile_csv_df
    
    # Reformatting steps (Warning: May take a few seconds.)
    turnstile_df.columns = turnstile_df.columns.str.replace(' ','') # Remove spaces in column names
    turnstile_df.columns = turnstile_df.columns.str.replace('/','') # Remove "/" in column names
    turnstile_df['DATETIMERAW'] = "" # Create new column called 'DATETIMERAW'
    turnstile_df['DATETIMERAW'] = turnstile_df.DATE + turnstile_df.TIME # Populate 'DATETIMERAW' with date and time concatenated string
    turnstile_df.DATETIMERAW = turnstile_df.DATETIMERAW.apply(lambda x : dt.datetime.strptime(x, "%m/%d/%Y%H:%M:%S")) # Convert DATETIMERAW into datetime object
    turnstile_df.TIME = turnstile_df.TIME.apply(lambda x : dt.datetime.strptime(x, "%H:%M:%S")) # Convert TIME into datetime object
    turnstile_df = turnstile_df.drop(columns = ['DATE']) # Drop DATE column

In [3]:
# Function grabs data at two individual time stamps and outputs the difference in the count between those timestamps
# This function references turnstile_df, which is the output of format_turnstile_df().
# Note that this function is not explicitly run, but instead is run within the 

def valuecount(timestamp1, timestamp2):
    """
    Timestamp inputs should be datetime objects. See example later. Also, timestamp2 should be greater than timestamp1.
    """
    global value_count_df
    
    # Grabbing the data
    turnstile_df1 = turnstile_df.loc[turnstile_df['DATETIMERAW'].isin([timestamp1])].reset_index(drop=True) # Grab dataset for timestamp1
    turnstile_df2 = turnstile_df.loc[turnstile_df['DATETIMERAW'].isin([timestamp2])].reset_index(drop=True) # Grab dataset for timestamp2
    
    # Merge two datasets, drop any rows that don't form a complete dataset
    turnstile_df_merge12 = turnstile_df1.merge(turnstile_df2.drop_duplicates(), on=["CA", "UNIT", "SCP", "STATION", "LINENAME", "DIVISION", "DESC"], how='outer').dropna().reset_index(drop=True)
    
    # Create a new dataframe with essential data
    turnstile_df_valuecounts = turnstile_df_merge12[["STATION"]] # Grabbing "STATION" name data
    turnstile_df_valuecounts["ENTRIES DIFFERENCE"] = (turnstile_df_merge12['ENTRIES_x'] - turnstile_df_merge12['ENTRIES_y']).abs() # Grabbing "ENTRIES" difference counts
    turnstile_df_valuecounts["EXITS DIFFERENCE"] = (turnstile_df_merge12['EXITS_x'] - turnstile_df_merge12['EXITS_y']).abs() # Grabbing "EXITS" difference counts
    
    # Computes the "ENTRIES" + "EXITS" for each counter
    turnstile_df_valuecounts["TOTAL"] = turnstile_df_valuecounts["ENTRIES DIFFERENCE"] + turnstile_df_valuecounts["EXITS DIFFERENCE"]  
    
    # Grab names of "STATION" and number of times they appear in turnstile_df_valuecounts
    station_count = Counter(turnstile_df_valuecounts['STATION'])
    station_dict = {i:station_count[i] for i in station_count}
    
    # For each "STATION" name, sum up all the counts from all units
    value_count_dict = dict(zip(list(station_dict.keys()), [[turnstile_df_valuecounts.loc[turnstile_df_valuecounts['STATION'] == i, 'TOTAL'].sum()] for i in list(station_dict.keys())]))
    
    # Grab list of most popular stations and puts it into a sorted dataframe column
    value_count_df = pd.DataFrame.from_dict(value_count_dict).transpose() # Grabbing values
    value_count_df.columns = ["ValueCount"] # Renaming column to ValueCount
    value_count_df = value_count_df.sort_values(by=["ValueCount"], ascending=False) # Sorting by count

In [199]:
# Defines the boundaries of the data pulled
# This function references turnstile_df, which is the output of format_turnstile_df().

def generate_timestamp_list():
    
    global timestamp_list
    datetimeraw_count = Counter(turnstile_df['DATETIMERAW'])
    
    i = 0
    timestamp_list = []
    while min(datetimeraw_count) + dt.timedelta(hours=4*i) < max(datetimeraw_count):
        timestamp_list.append(min(datetimeraw_count) + dt.timedelta(hours=4*i))
        i += 1

In [5]:
# Grabs for the whole week and enters it into a dataframe
# This function references the timestamp_list AND calls upon the valuecount function.

def grabweek():
    global value_count_df_1to2

    valuecount(timestamp_list[0], timestamp_list[1])
    value_count_df_1to2 = value_count_df[["ValueCount"]]
    value_count_df_1to2.columns = ["{}".format(timestamp_list[0])]

    for j in range(1, len(timestamp_list)-1):
        valuecount(timestamp_list[j], timestamp_list[j+1])
        value_count_df_1to2 = value_count_df_1to2.join(value_count_df[["ValueCount"]])
        value_count_df_1to2 = value_count_df_1to2.rename(columns={'ValueCount':"{}".format(str(timestamp_list[j]))})

In [6]:
# Import latlong_clean.csv, which was exported from a different notebook file.
# This is used at the end after running the pipeline

latlong_df = pd.read_csv(r"latlong_clean.csv")
latlong_df = latlong_df.drop(columns={"Unnamed: 0", "Numbering"}) # Dropping the "Unnamed: 0" and "Numbering" column
latlong_df.head() # Preview the data

,StationName,Latitude,Longitude
0,34 ST-HERALD SQ,40.749,-73.989
1,TIMES SQ-42 ST,40.755,-73.987
2,34 ST-PENN STA,40.751,-73.990
3,59 ST COLUMBUS,40.768,-73.982
4,86 ST,40.780,-73.956


In [34]:
# Grabbing turnstile filenames for the last year from the MTA website

URL = "http://web.mta.info/developers/turnstile.html"
soup = bs4.BeautifulSoup(urlopen(URL))

# Grabs all the links in the mta webpage
all_link_list = []
for link in soup.findAll('a'):
    all_link_list.append(link.get('href'))

# Dumps only the names for the last year into turnstile_link_list
turnstile_link_list = [all_link_list[36:89][i].replace("data/nyct/turnstile/", "") for i in range(len(link_list[36:89]))]
turnstile_link_list

['turnstile_190330.txt',
 'turnstile_190323.txt',
 'turnstile_190316.txt',
 'turnstile_190309.txt',
 'turnstile_190302.txt',
 'turnstile_190223.txt',
 'turnstile_190216.txt',
 'turnstile_190209.txt',
 'turnstile_190202.txt',
 'turnstile_190126.txt',
 'turnstile_190119.txt',
 'turnstile_190112.txt',
 'turnstile_190105.txt',
 'turnstile_181229.txt',
 'turnstile_181222.txt',
 'turnstile_181215.txt',
 'turnstile_181208.txt',
 'turnstile_181201.txt',
 'turnstile_181124.txt',
 'turnstile_181117.txt',
 'turnstile_181110.txt',
 'turnstile_181103.txt',
 'turnstile_181027.txt',
 'turnstile_181020.txt',
 'turnstile_181013.txt',
 'turnstile_181006.txt',
 'turnstile_180929.txt',
 'turnstile_180922.txt',
 'turnstile_180915.txt',
 'turnstile_180908.txt',
 'turnstile_180901.txt',
 'turnstile_180825.txt',
 'turnstile_180818.txt',
 'turnstile_180811.txt',
 'turnstile_180804.txt',
 'turnstile_180728.txt',
 'turnstile_180721.txt',
 'turnstile_180714.txt',
 'turnstile_180707.txt',
 'turnstile_180630.txt',


In [57]:
turnstile_link_list[25]

'turnstile_181006.txt'

In [58]:
"""
This loops through the turnstile_link_list and outputs the turnstile_YYMMDD_proc.csv files.

Notes:
1. Use the "20190405 Grabbing links from webpage" file to download the turnstile_YYMMDD.txt files from the MTA website "http://web.mta.info/developers/turnstile.html"
2. The turnstile_YYMMDD.txt files MUST be in the directory. If they are not, the loop will stop.
3. Keep an eye on the loop, it stops if there are any issues with the processing of the file
4. Files to skip: turnstile_190316.txt, turnstile_181110.txt
5. Note: See below for files that weren't processed correctly due to daylight savings

Warning: This may take a long time! On my computer, it outputs 2-3 files per minute and ran for 20 min+

"""

for i in turnstile_link_list[25:len(turnstile_link_list)]: # Note, if loop stops, restart at an appropriate lowerbound for range

    turnstile_df = pd.read_csv(i) # File imports
    format_turnstile_df(turnstile_df) # Warning: Run once per file import ONLY

    generate_timestamp_list() # Generating list of timestamps
    grabweek() # Warning: May take a minute
    value_count_df_1to2 = value_count_df_1to2.reset_index().rename(columns={"index":"StationName"})

    # Merge coordinate data with value_count data
    value_count_df_1to2_with_coord = latlong_df.merge(value_count_df_1to2, on=["StationName"], how='left').reset_index(drop=True)
    value_count_df_1to2_with_coord.head() # Preview the final data

    value_count_df_1to2_with_coord.to_csv("{}_proc.csv".format(i.replace(".txt","")))
    
# Note: Graphing part was moved to next notebook

In [88]:
"""
The following is for AFTER running the above loop and downloading as many files as possible
"""
turnstile_proc_list = [turnstile_link_list[i].replace(".txt", "")+"_proc.csv" for i in range(len(turnstile_link_list))]
turnstile_proc_list.pop(2) # Removing 'turnstile_190316_proc.csv' from list
turnstile_proc_list.pop(19) # Removing 'turnstile_181110_proc.csv' from list
turnstile_proc_list # Show the list of files

['turnstile_190330_proc.csv',
 'turnstile_190323_proc.csv',
 'turnstile_190309_proc.csv',
 'turnstile_190302_proc.csv',
 'turnstile_190223_proc.csv',
 'turnstile_190216_proc.csv',
 'turnstile_190209_proc.csv',
 'turnstile_190202_proc.csv',
 'turnstile_190126_proc.csv',
 'turnstile_190119_proc.csv',
 'turnstile_190112_proc.csv',
 'turnstile_190105_proc.csv',
 'turnstile_181229_proc.csv',
 'turnstile_181222_proc.csv',
 'turnstile_181215_proc.csv',
 'turnstile_181208_proc.csv',
 'turnstile_181201_proc.csv',
 'turnstile_181124_proc.csv',
 'turnstile_181117_proc.csv',
 'turnstile_181103_proc.csv',
 'turnstile_181027_proc.csv',
 'turnstile_181020_proc.csv',
 'turnstile_181013_proc.csv',
 'turnstile_181006_proc.csv',
 'turnstile_180929_proc.csv',
 'turnstile_180922_proc.csv',
 'turnstile_180915_proc.csv',
 'turnstile_180908_proc.csv',
 'turnstile_180901_proc.csv',
 'turnstile_180825_proc.csv',
 'turnstile_180818_proc.csv',
 'turnstile_180811_proc.csv',
 'turnstile_180804_proc.csv',
 'turnstil

In [ ]:
i = 0

In [189]:
# Loop through the files just to inspect them

turnstile_proc_df = pd.read_csv(turnstile_proc_list[i])
i += 1
print(turnstile_proc_list[i])
turnstile_proc_df.head()

turnstile_180331_proc.csv


,Unnamed: 0,StationName,Latitude,Longitude,2018-03-31 00:00:00,2018-03-31 04:00:00,2018-03-31 08:00:00,2018-03-31 12:00:00,2018-03-31 16:00:00,2018-03-31 20:00:00,2018-04-01 00:00:00,2018-04-01 04:00:00,2018-04-01 08:00:00,2018-04-01 12:00:00,2018-04-01 16:00:00,2018-04-01 20:00:00,2018-04-02 00:00:00,2018-04-02 04:00:00,2018-04-02 08:00:00,2018-04-02 12:00:00,2018-04-02 16:00:00,2018-04-02 20:00:00,2018-04-03 00:00:00,2018-04-03 04:00:00,2018-04-03 08:00:00,2018-04-03 12:00:00,2018-04-03 16:00:00,2018-04-03 20:00:00,2018-04-04 00:00:00,2018-04-04 04:00:00,2018-04-04 08:00:00,2018-04-04 12:00:00,2018-04-04 16:00:00,2018-04-04 20:00:00,2018-04-05 00:00:00,2018-04-05 04:00:00,2018-04-05 08:00:00,2018-04-05 12:00:00,2018-04-05 16:00:00,2018-04-05 20:00:00,2018-04-06 00:00:00,2018-04-06 04:00:00,2018-04-06 08:00:00,2018-04-06 12:00:00,2018-04-06 16:00:00
0,0,34 ST-HERALD SQ,40.749,-73.989,4532.0,4123.0,20716.0,35526.0,42835.0,23544.0,4142.0,2795.0,14105.0,25381.0,28065.0,15201.0,1692.0,15036.0,56311.0,42998.0,35156.0,8949.0,2276.0,16455.0,61994.0,50253.0,89444.0,26671.0,2365.0,16632.0,61600.0,49070.0,89805.0,29155.0,2651.0,16787.0,64012.0,52793.0,92516.0,31951.0,3058.0,15463.0,57141.0,49037.0,61982.0
1,1,TIMES SQ-42 ST,40.755,-73.987,7059.0,4624.0,18916.0,30866.0,39834.0,27644.0,6664.0,3089.0,13137.0,24047.0,29613.0,18873.0,3339.0,10566.0,43160.0,31246.0,60908.0,22856.0,3887.0,13138.0,49463.0,37650.0,68801.0,25810.0,3872.0,13173.0,51481.0,41542.0,71545.0,29084.0,4150.0,13272.0,50084.0,39891.0,70466.0,30385.0,4902.0,12338.0,45113.0,37819.0,65766.0
2,2,34 ST-PENN STA,40.751,-73.990,4174.0,5857.0,24925.0,37166.0,42072.0,21916.0,4171.0,3663.0,18157.0,27754.0,30766.0,17532.0,1989.0,22581.0,58068.0,35609.0,67799.0,17617.0,2174.0,24170.0,60744.0,41802.0,80724.0,21830.0,2267.0,24763.0,59834.0,37266.0,77612.0,23194.0,2594.0,23685.0,59926.0,39396.0,79740.0,24825.0,2788.0,21365.0,54956.0,41903.0,75554.0
3,3,59 ST COLUMBUS,40.768,-73.982,2739.0,3098.0,15095.0,25772.0,26450.0,11733.0,2543.0,2291.0,11537.0,20157.0,19778.0,8129.0,1228.0,7860.0,26649.0,23883.0,35633.0,13121.0,1770.0,9032.0,30781.0,27010.0,39996.0,14138.0,1819.0,9079.0,31241.0,27830.0,41421.0,15358.0,1986.0,9285.0,32173.0,30709.0,43145.0,17289.0,2139.0,8575.0,28754.0,27253.0,39519.0
4,4,86 ST,40.780,-73.956,1819.0,3898.0,14186.0,21392.0,22216.0,10452.0,1642.0,2524.0,10046.0,16476.0,14785.0,6711.0,905.0,10106.0,27184.0,22380.0,32850.0,9197.0,841.0,11849.0,30623.0,26937.0,36701.0,10484.0,847.0,11816.0,30735.0,26211.0,36524.0,11003.0,881.0,11628.0,29097.0,25704.0,37098.0,11576.0,980.0,10762.0,27480.0,25123.0,35810.0


In [194]:
# The following items have a lot of missing values due to daylight savings.
# See adjustment in generate_timestamp_list function below
turnstile_link_list[3:21]

['turnstile_190316.txt',
 'turnstile_190309.txt',
 'turnstile_190302.txt',
 'turnstile_190223.txt',
 'turnstile_190216.txt',
 'turnstile_190209.txt',
 'turnstile_190202.txt',
 'turnstile_190126.txt',
 'turnstile_190119.txt',
 'turnstile_190112.txt',
 'turnstile_190105.txt',
 'turnstile_181229.txt',
 'turnstile_181222.txt',
 'turnstile_181215.txt',
 'turnstile_181208.txt',
 'turnstile_181201.txt',
 'turnstile_181124.txt',
 'turnstile_181117.txt',
 'turnstile_181110.txt']

In [223]:
# Defines the boundaries of the data pulled
# This function references turnstile_df, which is the output of format_turnstile_df().

def generate_timestamp_list_dst():
    
    global timestamp_list
    datetimeraw_count = Counter(turnstile_df['DATETIMERAW'])
    
    i = 0
    timestamp_list = []
    while min(datetimeraw_count) + dt.timedelta(hours=4*i) < max(datetimeraw_count):
        timestamp_list.append(min(datetimeraw_count) + dt.timedelta(hours=4*i+3))
        i += 1

In [225]:
"""
This loops through the turnstile_link_list and outputs the turnstile_YYMMDD_proc.csv files.
This segment is just for the dates that are within (the opposite of) DST
Warning! turnstile_190316.txt, turnstile_181110.txt still cannot be processed correctly and should be removed.

Notes:
1. Use the "20190405 Grabbing links from webpage" file to download the turnstile_YYMMDD.txt files from the MTA website "http://web.mta.info/developers/turnstile.html"
2. The turnstile_YYMMDD.txt files MUST be in the directory. If they are not, the loop will stop.
3. Keep an eye on the loop, it stops if there are any issues with the processing of the file

Warning: This may take a long time! On my computer, it outputs 2-3 files per minute and ran for 10+ min.

"""

for i in turnstile_link_list[3:21]: # Note, if loop stops, restart at an appropriate lowerbound for range

    turnstile_df = pd.read_csv(i) # File imports
    format_turnstile_df(turnstile_df) # Warning: Run once per file import ONLY

    generate_timestamp_list_dst() # Generating list of timestamps
    grabweek() # Warning: May take a minute
    value_count_df_1to2 = value_count_df_1to2.reset_index().rename(columns={"index":"StationName"})

    # Merge coordinate data with value_count data
    value_count_df_1to2_with_coord = latlong_df.merge(value_count_df_1to2, on=["StationName"], how='left').reset_index(drop=True)
    value_count_df_1to2_with_coord.head() # Preview the final data

    value_count_df_1to2_with_coord.to_csv("{}_proc.csv".format(i.replace(".txt","")))
    
# Note: Graphing part was moved to next notebook

ValueError: Length mismatch: Expected axis has 0 elements, new values have 1 elements